In [1]:
import numpy as np
import pandas as pd

In [15]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [17]:
target = 'category'

In [16]:
train.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


In [ ]:
# lbl_enc = preprocessing.LabelEncoder()
# y = lbl_enc.fit_transform(train.author.values)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [71]:
svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

from sklearn.decomposition import TruncatedSVD
lsi = TruncatedSVD(n_components=100,
                  algorithm='randomized',
                  n_iter=100)
#vect = TfidfVectorizer(stop_words='english')
vect = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
            stop_words = 'english')

clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect),('lsi', lsi),  ('clf', clf)])

params = {
    'lsi__n_components': [10,100,250],
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(5, 10, 15,20)
}

In [72]:
search = RandomizedSearchCV(pipe, params, cv=4, n_jobs=6, verbose=1)

In [83]:
search.fit(train['description'], train[target])

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


KeyboardInterrupt: 

In [76]:
search.best_score_

0.8758700696055685

In [77]:
search.best_params_

{'vect__max_features': 1000,
 'vect__max_df': 1.0,
 'lsi__n_components': 10,
 'clf__n_estimators': 10,
 'clf__max_depth': 10}

In [51]:
y_proba = search.predict_proba(train['description'])

In [63]:
y_proba.shape

(2586, 4)

In [57]:
results = pd.DataFrame({'predict' : y_proba[:,0],
                        'predict_proba' : y_proba[:,1],
                        'actual'  : train['category']})

In [58]:
results.head()

,predict,predict_proba,actual
0,0.1,0.9,2
1,1.0,0.0,1
2,0.0,1.0,2
3,0.7,0.0,1
4,0.0,1.0,2


In [44]:
results.actual.shape[0]

2586

In [91]:
# Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

#pipe = Pipeline([('vect', vect),('lsi', lsi),
mylsi = Pipeline([('vect', vect),('lsi', lsi)])
pipe = Pipeline([('mylsi', mylsi),('clf', clf)])

In [93]:
result = mylsi.fit_transform(train['description'], train[target])

In [101]:
from sklearn.model_selection import cross_val_predict
cross_val_predict(MultinomialNB(), train['description'], train[target], cv=3)

ValueError: could not convert string to float: 'Gorgeous dried apricot, peach cobbler, vanilla custard, plus a hint of cherry-flavored pipe tobacco. Rich, layered, and rounded caramel, with hints of cinnamon, nutmeg, clove, and allspice. Then earth, and lots of it, over a slightly chewy mouthfeel of caramel chew and Red Hots cinnamon candy. Medium finish counters wonderful bouquet and palate notes, but still lovely for sipping.'

In [ ]:
# pipe.predict(train['description'])